In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from math import sqrt

In [2]:
# Specify data types for columns in books and users datasets
dtype_books = {
    'isbn': 'str',
    'book_title': 'str',
    'book_author': 'str',
    'year_of_publication': 'str',
    'publisher': 'str'
}

dtype_users = {
    'user_id': 'str',
    'Location': 'str',
    'Age': 'float'
}

# Load datasets with specified data types
books = pd.read_csv('BX-Books.csv', dtype=dtype_books, low_memory=False)
users = pd.read_csv('BX-Users.csv', dtype=dtype_users, low_memory=False)
ratings = pd.read_csv('BX-Book-Ratings.csv')

In [3]:
users.head()

,user_id,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [4]:
users.shape

(278859, 3)

In [5]:
books.shape

(271379, 5)

In [6]:
ratings.shape

(1048575, 3)

In [7]:
books.head()

,isbn,book_title,book_author,year_of_publication,publisher
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company


In [8]:
books.isnull().sum()

isbn                   0
book_title             0
book_author            2
year_of_publication    0
publisher              2
dtype: int64

In [9]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271377 non-null  object
 3   year_of_publication  271379 non-null  object
 4   publisher            271377 non-null  object
dtypes: object(5)
memory usage: 10.4+ MB


In [10]:
# Convert 'year_of_publication' to numeric, setting errors='coerce' will convert invalid parsing to NaN
books['year_of_publication'] = pd.to_numeric(books['year_of_publication'], errors='coerce')

# Fill NaN values with the mode of the column or a specific value
books['year_of_publication'].fillna(books['year_of_publication'].mode()[0], inplace=True)

# Convert the column to integer type
books['year_of_publication'] = books['year_of_publication'].astype(int)
books['isbn']=books['isbn'].astype(str)

In [11]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271377 non-null  object
 3   year_of_publication  271379 non-null  int32 
 4   publisher            271377 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.3+ MB


In [12]:
books['year_of_publication'].replace('0', books['year_of_publication'].median(), inplace=True)
books.fillna({'publisher': 'Unknown'}, inplace=True)
books.fillna({'book_author': 'Unknown'}, inplace=True)

In [13]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271379 non-null  object
 3   year_of_publication  271379 non-null  int32 
 4   publisher            271379 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.3+ MB


In [14]:
books.isnull().sum()

isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
dtype: int64

In [15]:
users.isnull().sum()

user_id          0
Location         1
Age         110763
dtype: int64

In [16]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278859 entries, 0 to 278858
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   278859 non-null  object 
 1   Location  278858 non-null  object 
 2   Age       168096 non-null  float64
dtypes: float64(1), object(2)
memory usage: 6.4+ MB


In [17]:
users['Age'].fillna(users['Age'].median(),inplace=True)
users['Location'].fillna('Unknown',inplace=True)

In [18]:
users.isnull().sum()

user_id     0
Location    0
Age         0
dtype: int64

In [19]:
users.describe()

,Age
count,278859.000000
mean,33.658562
std,11.282598
min,0.000000
25%,29.000000
50%,32.000000
75%,35.000000
max,244.000000


In [20]:
# Data Cleaning
users = users[(users['Age'] >= 5) & (users['Age'] <= 100)]

In [21]:
users.describe()

,Age
count,277611.000000
mean,33.650767
std,10.654258
min,5.000000
25%,29.000000
50%,32.000000
75%,35.000000
max,100.000000


In [22]:
ratings.isnull().sum()

user_id    0
isbn       0
rating     0
dtype: int64

In [23]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271379 non-null  object
 3   year_of_publication  271379 non-null  int32 
 4   publisher            271379 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.3+ MB


In [24]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1048575 non-null  int64 
 1   isbn     1048575 non-null  object
 2   rating   1048575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [25]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277611 entries, 0 to 278858
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   277611 non-null  object 
 1   Location  277611 non-null  object 
 2   Age       277611 non-null  float64
dtypes: float64(1), object(2)
memory usage: 8.5+ MB


In [26]:
users.duplicated().sum()

0

In [27]:
books.duplicated().sum()

0

In [28]:
ratings.duplicated().sum()

146

In [29]:
# Step 1: Identify non-numeric values in users dataframe
print("Non-numeric user_id values:")
print(users[~users['user_id'].apply(lambda x: x.isnumeric() if pd.notnull(x) else True)])

Non-numeric user_id values:
                user_id Location   Age
275081  , milan, italy"  Unknown  32.0


In [30]:
# Step 3: Convert to numeric, coercing errors to NaN
users['user_id'] = pd.to_numeric(users['user_id'], errors='coerce')

In [31]:
# Step 4: Handle NaN values resulting from coercion (e.g., drop or fill)
users.dropna(subset=['user_id'], inplace=True)  # Drop rows with NaNs in 'user_id'

In [32]:
users.isnull().sum()

user_id     0
Location    0
Age         0
dtype: int64

In [33]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277610 entries, 0 to 278858
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   user_id   277610 non-null  float64
 1   Location  277610 non-null  object 
 2   Age       277610 non-null  float64
dtypes: float64(2), object(1)
memory usage: 8.5+ MB


In [34]:
users['user_id']=users['user_id'].astype('int64')
users['Age']=users['Age'].astype('int64')

In [35]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277610 entries, 0 to 278858
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   user_id   277610 non-null  int64 
 1   Location  277610 non-null  object
 2   Age       277610 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 8.5+ MB


In [36]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1048575 non-null  int64 
 1   isbn     1048575 non-null  object
 2   rating   1048575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [37]:
# Number of unique users and books
num_unique_users = ratings['user_id'].nunique()
num_unique_books = ratings['isbn'].nunique()
print(f'Number of unique users: {num_unique_users}')
print(f'Number of unique books: {num_unique_books}')

Number of unique users: 95513
Number of unique books: 322102


In [38]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271379 non-null  object
 3   year_of_publication  271379 non-null  int32 
 4   publisher            271379 non-null  object
dtypes: int32(1), object(4)
memory usage: 9.3+ MB


In [39]:
# Convert ISBN to numeric numbers in the correct order
isbn_to_numeric = {isbn: idx for idx, isbn in enumerate(books['isbn'].unique())}
books['ISBN_numeric'] = books['isbn'].map(isbn_to_numeric)

In [40]:
# Convert user_id to numeric numbers in the correct order
user_to_numeric = {user: idx for idx, user in enumerate(users['user_id'].unique())}
users['User_ID_numeric'] = users['user_id'].map(user_to_numeric)

In [41]:
books.head(10)

,isbn,book_title,book_author,year_of_publication,publisher,ISBN_numeric
0,195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,0
1,2005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,1
2,60973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,2
3,374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,3
4,393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,4
5,399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,5
6,425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,6
7,671870432,PLEADING GUILTY,Scott Turow,1993,Audioworks,7
8,679425608,Under the Black Flag: The Romance and the Real...,David Cordingly,1996,Random House,8
9,074322678X,Where You'll Find Me: And Other Stories,Ann Beattie,2002,Scribner,9


In [42]:
users.head(10)

,user_id,Location,Age,User_ID_numeric
0,1,"nyc, new york, usa",32,0
1,2,"stockton, california, usa",18,1
2,3,"moscow, yukon territory, russia",32,2
3,4,"porto, v.n.gaia, portugal",17,3
4,5,"farnborough, hants, united kingdom",32,4
5,6,"santa monica, california, usa",61,5
6,7,"washington, dc, usa",32,6
7,8,"timmins, ontario, canada",32,7
8,9,"germantown, tennessee, usa",32,8
9,10,"albacete, wisconsin, spain",26,9


In [43]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   isbn                 271379 non-null  object
 1   book_title           271379 non-null  object
 2   book_author          271379 non-null  object
 3   year_of_publication  271379 non-null  int32 
 4   publisher            271379 non-null  object
 5   ISBN_numeric         271379 non-null  int64 
dtypes: int32(1), int64(1), object(4)
memory usage: 11.4+ MB


In [44]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 277610 entries, 0 to 278858
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   user_id          277610 non-null  int64 
 1   Location         277610 non-null  object
 2   Age              277610 non-null  int64 
 3   User_ID_numeric  277610 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 10.6+ MB


In [45]:
# Convert both user_id and ISBN to the ordered list in the ratings dataframe
ratings['User_ID_numeric'] = ratings['user_id'].map(user_to_numeric)
ratings['ISBN_numeric'] = ratings['isbn'].map(isbn_to_numeric)

In [46]:
ratings.head(50)

,user_id,isbn,rating,User_ID_numeric,ISBN_numeric
0,276725,034545104X,0,275483.0,2966.0
1,276726,155061224,5,275484.0,225829.0
2,276727,446520802,0,275485.0,11054.0
3,276729,052165615X,3,275487.0,246854.0
4,276729,521795028,6,275487.0,246855.0
5,276733,2080674722,0,275491.0,123645.0
6,276736,3257224281,8,275494.0,NaN
7,276737,600570967,6,275495.0,NaN
8,276744,038550120X,7,275502.0,9295.0
9,276745,342310538,10,275503.0,NaN


In [47]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 5 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   user_id          1048575 non-null  int64  
 1   isbn             1048575 non-null  object 
 2   rating           1048575 non-null  int64  
 3   User_ID_numeric  1042950 non-null  float64
 4   ISBN_numeric     941148 non-null   float64
dtypes: float64(2), int64(2), object(1)
memory usage: 40.0+ MB


In [48]:
ratings.isnull().sum()

user_id                 0
isbn                    0
rating                  0
User_ID_numeric      5625
ISBN_numeric       107427
dtype: int64

In [49]:
ratings['User_ID_numeric'].fillna(ratings['User_ID_numeric'].mean(), inplace=True)

In [50]:
ratings['ISBN_numeric'].fillna(ratings['ISBN_numeric'].mean(), inplace=True)

In [51]:
ratings.isnull().sum()

user_id            0
isbn               0
rating             0
User_ID_numeric    0
ISBN_numeric       0
dtype: int64

# Re-index the columns to build a matrix
ratings_matrix = ratings.pivot(index='User_ID_numeric', columns='ISBN_numeric', values='rating').fillna(0)

In [53]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

In [54]:
# Making predictions (simple example using mean rating for prediction)
# Note: This is a very basic example of a prediction model
mean_rating = train_data.groupby('User_ID_numeric')['rating'].mean()
predictions = test_data.copy()
predictions['predicted_rating'] = test_data['User_ID_numeric'].map(mean_rating)

In [55]:
predictions.head()

,user_id,isbn,rating,User_ID_numeric,ISBN_numeric,predicted_rating
781974,189334,874776945,0,188491.0,5503.0,2.431410
937737,227447,345435249,0,226423.0,1351.0,0.661874
907828,220586,888947216,9,219595.0,91205.0,4.500000
784628,189835,553234994,5,188988.0,172938.0,4.932741
662460,160735,61098760,0,160022.0,13442.0,1.782051


In [56]:
predictions.isnull().sum()

user_id                 0
isbn                    0
rating                  0
User_ID_numeric         0
ISBN_numeric            0
predicted_rating    11787
dtype: int64

In [57]:
# Fill missing predictions with the overall mean rating
overall_mean_rating = train_data['rating'].mean()
predictions['predicted_rating'].fillna(overall_mean_rating, inplace=True)

In [58]:
predictions.isnull().sum()

user_id             0
isbn                0
rating              0
User_ID_numeric     0
ISBN_numeric        0
predicted_rating    0
dtype: int64

In [59]:
# Evaluate the predictions using RMSE
rmse = sqrt(mean_squared_error(test_data['rating'], predictions['predicted_rating']))
print(f'RMSE: {rmse}')

RMSE: 3.4378993143595578
